# CNN to solve a multiband linear equation system

In [1]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader

# from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Samples Generator

In [2]:
def tensor2matrix(t):
    ni, nj = t[0].shape

    diag_ni_u = torch.flatten(t[0].transpose(0,1))
    diag_1_u = torch.flatten(t[1].transpose(0,1))
    diag_1_l = torch.flatten(t[2].transpose(0,1))
    diag_ni_l = torch.flatten(t[3].transpose(0,1))

    n_diag = ni*nj
    diag = [1.]*n_diag
    diag = torch.tensor(diag).float()

    m = torch.zeros((n_diag,n_diag))
    m = m + torch.diag_embed(diag)
    m[1:,:-1] = m[1:,:-1] + torch.diag_embed(diag_1_l[:-1])
    m[:-1,1:] = m[:-1,1:] + torch.diag_embed(diag_1_u[1:])
    m[ni:,:-ni] = m[ni:,:-ni] + torch.diag_embed(diag_ni_l[:-ni])
    m[:-ni,ni:] = m[:-ni,ni:] + torch.diag_embed(diag_ni_u[ni:])

    return m

def make_tensor(number_generator,
                args={},
                symmetric=True,
                off_diagonal_abs_mean=0.5,
                grid_size=(5, 5)):

    #Generate values
    ni, nj = grid_size
    n_diag_1 = (ni-1)*nj
    n_diag_ni = ni*(nj-1)

    diag_1_u = [number_generator(**args) for _ in range(n_diag_1)]
    diag_ni_u = [number_generator(**args) for _ in range(n_diag_ni)]

    if symmetric:
        diag_1_l = diag_1_u
        diag_ni_l = diag_ni_u
    else:
        diag_1_l = [number_generator(**args) for _ in range(n_diag_1)]
        diag_ni_l = [number_generator(**args) for _ in range(n_diag_ni)]

    #To tensor
    diag_1_u = torch.tensor(diag_1_u).float()
    diag_1_l = torch.tensor(diag_1_l).float()
    diag_ni_u = torch.tensor(diag_ni_u).float()
    diag_ni_l = torch.tensor(diag_ni_l).float()

    #Scale off main diagonal
    off_diagonal = torch.cat([diag_ni_u, diag_1_u, diag_1_l, diag_ni_l])
    off_diagonal = torch.abs(off_diagonal)
    mean_abs = torch.mean(off_diagonal)
    alpha = off_diagonal_abs_mean / mean_abs
    diag_1_u = torch.mul(diag_1_u, alpha)
    diag_1_l = torch.mul(diag_1_l, alpha)
    diag_ni_u = torch.mul(diag_ni_u, alpha)
    diag_ni_l = torch.mul(diag_ni_l, alpha)

    #Adjust zeroes
    diag_1_u = torch.reshape(diag_1_u, (ni-1,nj))
    diag_1_u = torch.cat([torch.zeros(1,nj), diag_1_u], dim=0)

    diag_1_l = torch.reshape(diag_1_l, (ni-1,nj))
    diag_1_l = torch.cat([diag_1_l, torch.zeros(1,nj)], dim=0)

    diag_ni_l = torch.reshape(diag_ni_l, (ni,nj-1))
    diag_ni_l = torch.cat([diag_ni_l, torch.zeros(ni,1)], dim=1)

    diag_ni_u = torch.reshape(diag_ni_u, (ni,nj-1))
    diag_ni_u = torch.cat([torch.zeros(ni,1), diag_ni_u], dim=1)

    t = torch.cat([diag_ni_u.unsqueeze(0),
                    diag_1_u.unsqueeze(0),
                    diag_1_l.unsqueeze(0),
                    diag_ni_l.unsqueeze(0)],
                    dim=0)
    return t

In [3]:
def get_sample(number_generator,
               args={},
               symmetric=True,
               off_diagonal_abs_mean=0.5,
               grid_size=(5, 5)):

    A = make_tensor(number_generator=number_generator,
                    args=args,
                    symmetric=symmetric,
                    off_diagonal_abs_mean=off_diagonal_abs_mean,
                    grid_size=grid_size)

    ni, nj = grid_size
    n_diag = ni*nj
    x_true = [number_generator(**args) for _ in range(n_diag)]
    x_true = torch.tensor(x_true).float()

    A_mat = tensor2matrix(A)
    b = torch.matmul(A_mat, x_true)
    b = torch.reshape(b, (nj,ni)).transpose(0,1)

    X = torch.cat([A, b.unsqueeze(0)], dim=0)
    y = torch.reshape(x_true, (nj,ni)).transpose(0,1)
    return (X, y)

In [4]:
def check_sample(X, y, result='tensor', print_cond_number=False):

    def _error_from_y(X,y):
        custom_weights = np.array([
            [[0, 0, 0], [0, 1, 0], [0, 0, 0]],
            [[0, 0, 0], [0, 0, 1], [0, 0, 0]],
            [[0, 0, 0], [0, 0, 0], [0, 1, 0]],
            [[0, 1, 0], [0, 0, 0], [0, 0, 0]],
            [[0, 0, 0], [1, 0, 0], [0, 0, 0]],
            ], dtype=np.float32)
        custom_weights_tensor = torch.tensor(custom_weights).unsqueeze(0)  # Add batch dimensions
        conv_layer = nn.Conv2d(in_channels=5, out_channels=1, kernel_size=3, stride=1, padding=1, bias=False)
        conv_layer.weight = nn.Parameter(custom_weights_tensor, requires_grad=False)

        ni, nj = X[0].shape
        X_ = torch.cat([torch.ones((ni,nj)).unsqueeze(0), X[:4]])
        residuals = conv_layer(X_ * y) - X[-1]
        return residuals

    def _error_from_solve(X):
        A = tensor2matrix(X[:4])
        b = torch.flatten(X[-1].transpose(0,1))
        y_aprox = np.linalg.solve(A, b)
        y_aprox = torch.tensor(y_aprox)
        residuals = torch.matmul(A, y_aprox) - b
        return residuals, y_aprox

    def _get_result(output_tensor):
        if result == 'tensor':
            return output_tensor
        if result == 'sum_abs':
            return float(torch.sum(torch.abs(output_tensor)))
        if result == 'max':
            return float(torch.max(torch.abs(output_tensor)))
        if result == 'sum2':
            return float(torch.sum(torch.square(output_tensor)))
        raise ValueError(f'Invalid option: {result}.')


    if print_cond_number:
        A = tensor2matrix(X[:4])
        cond_num = np.linalg.cond(A)
        print(f'Matrix condition number: {cond_num:0.4g}')
    residuals = _error_from_y(X,y)
    print(f'Result using provided y: {_get_result(residuals):0.4g}')
    residuals, y_aprox = _error_from_solve(X)
    print(f'Result using numpy calculated y: {_get_result(residuals):0.4g}')
    y_ = torch.flatten(y.transpose(0,1))
    print(f'Diffence in y vectors: {_get_result(y_ - y_aprox):0.4g}')

In [5]:
ni, nj = 100, 100
number_generator = random.uniform
args = {'a':-1,'b':3}

In [6]:
m = make_tensor(
    number_generator=number_generator,
    args=args,
    grid_size=(ni,nj),
    off_diagonal_abs_mean=2,
    symmetric=False)
print(m.shape)
print(tensor2matrix(m).shape)

torch.Size([4, 100, 100])
torch.Size([10000, 10000])


In [7]:
X, y = get_sample(
    number_generator=number_generator,
    args=args,
    # one,
    grid_size=(ni,nj),
    off_diagonal_abs_mean=0.2,
    symmetric=False)
print(X.shape)
print(y.shape)

torch.Size([5, 100, 100])
torch.Size([100, 100])


In [8]:
check_sample(X,y, 'max')

Result using provided y: 4.768e-07
Result using numpy calculated y: 4.768e-07
Diffence in y vectors: 9.537e-07
